# Ejemplo de red neuronal para clasificación utilizando el dataset del Titanic

## Pasos iniciales

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

#from sklearn.datasets import load_iris

from sklearn.metrics import mean_squared_error

from tensorflow.keras.optimizers import Adam

## Cargamos el dataset

In [ ]:
ruta = "https://raw.githubusercontent.com/vbatiz/intro-python/main/notebooks/data/titanic.csv"
df_titanic = pd.read_csv(ruta)
df_titanic.head()

## Procesamos los datos

Eliminamos los registros con datos nulos

In [ ]:
df_titanic = df_titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived']].dropna()

In [ ]:
df_titanic.columns

In [ ]:
df_titanic.head()

### Convertimos la columna [Sex] a su representación numérica

In [ ]:
df_titanic['Sex'] = df_titanic['Sex'].map({'male': 0, 'female': 1})

### Elegimos las variables predictoras (X's)

In [ ]:
X = df_titanic[['Pclass', 'Sex', 'Age']]

### Separamos la variable objetivo

In [ ]:
y = df_titanic['Survived']

### Estandarizamos los atributos (features) seleccionados

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Preparamos el modelo

### Separamos el dataset en entrenamiento y prueba

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=34)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=34)


In [ ]:
X_train.shape

### Construimos el modelo

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification loss function
              metrics=['accuracy'])

### Entrenamos el modelo

In [ ]:
epocas = 100
tamanio_lote = 32
history = model.fit(X_train, y_train, epochs=epocas, batch_size=tamanio_lote, validation_data=(X_val, y_val))

### Realizamos predicciones en el conjunto de datos de prueba

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

### Evaluamos el modelo

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"\nExactitud (Accuracy): {accuracy*100:.2f}%")

### Graficamos la historia de entrenamiento

In [ ]:
# Graficamos los valores de exactitud (accuracy) para entrenamiento y validación 
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Exactitud (accuracy) del Modelo')
plt.xlabel('Época')
plt.ylabel('Exactitud (Accuracy)')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')

# Graficamos los valores de pérdida (loss) para entrenamiento y validación 
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdida (loss) del Modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida (Loss)')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')

plt.tight_layout()
plt.show()